# 查看有监督微调的数据集格式

In [1]:
import json

pretrain_dataset_path = r"D:\MiniMind\dataset\sft_mini_512.jsonl"
with open(pretrain_dataset_path, "r", encoding="utf-8") as f:
    for line_num, line in enumerate(f,1):
        data = json.loads(line.strip())
        break

print(data.keys())
print(data)

dict_keys(['conversations'])
{'conversations': [{'role': 'user', 'content': '请告诉我在中国古代的“四大发明”是什么？'}, {'role': 'assistant', 'content': '中国古代的“四大发明”是指造纸术、印刷术、火药和指南针。这四项发明对世界文明的发展产生了深远的影响：\n\n1. **造纸术**：据史书记载，东汉时期的蔡伦改进了造纸工艺，使得纸张的生产更加便捷、成本更低，质量也更加优良。这一发明极大地促进了文化的传播和保存。\n\n2. **印刷术**：中国古代的印刷术最早可以追溯到唐代的雕版印刷，到了宋代发展出了活字印刷技术。印刷术的发明极大地促进了知识的传播，降低了书籍的成本，对教育和文化的发展起到了重要的推动作用。\n\n3. **火药**：火药最初是在唐代被发现的，最初可能用于医疗或炼金术。到了宋代，人们开始将火药用于军事目的，发明了各种火器。火药的发明改变了战争的面貌，对世界军事技术的发展产生了重大影响。\n\n4. **指南针**：指南针最初在中国被用于风水测量，后来逐渐发展成为航海导航的重要工具。这一发明极大地促进了海上航行技术的发展，对于新航路的开辟和世界地理大发现起到了关键作用。\n\n这四项发明不仅在中国历史上占有重要地位，而且对全世界的科技进步和文明发展都产生了深远的影响。'}]}


# 有监督微调数据加载器

In [2]:
import json
import torch
from torch.utils.data import Dataset

class SFTDataset(Dataset):
    def __init__(self, jsonl_path, tokenizer, max_length=1024):
        super().__init__()
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.samples = self.load_data(jsonl_path)
        # tokenizer把文本转换成token id
        # input_ids表示分词后得到的token id序列
        # add_special_tokens=False表示不添加特殊字符
        self.bos_id = tokenizer('<im_start>assistant', add_special_tokens=False).input_ids
        self.eos_id = tokenizer('<im_end>', add_special_tokens=False).input_ids
    def __len__(self):
        return len(self.samples) # 返回样本数量
    def load_data(self, path):
        # 从jsonl文件加载对话数据
        samples = []
        with open(path, 'r', encoding='utf-8') as f:
            # 行号默认从1开始
            for line_num, line in enumerate(f, 1):
                data = json.loads(line.strip()) # 每行为一个JSON对象
                samples.append(data)
        return samples
    def _create_chat_prompt(self, conversations):
        # 对话轮构成符合ChatML格式的字符串，每一轮用户、助手对话被标注为'user'和'assistant'
        messages = []
        for i, turn in enumerate(conversations):
            role = 'user' if i % 2 == 0 else 'assistant' # 偶数轮为用户，奇数轮为助手
            messages.append({"role": role, "content": turn["content"]})
        # 返回字符串形式的prompt,而非直接tokenize
        return self.tokenizer.apply_chat_template(
            messages, # 结构化对话列表
            tokenize=False, # 是否分词
            add_generation_prompt=False # 是否在末尾自动添加一个"assistant开始说话"的提示
        )
    def _generate_loss_mask(self, input_ids):
        # 构建损失掩码，只有assistant的回答部分才参与loss计算
        # 找出每一段assistant的响应，在其<im_start>assistant和<im_end>之间设置loss_mask为1
        loss_mask = [0] * len(input_ids)
        i = 0
        while i < len(input_ids):
            # 找assistant开头标志
            if input_ids[i: i+len(self.bos_id)] == self.bos_id:
                start = i + len(self.bos_id) # 答案起点
                end = start
                while end < len(input_ids):
                    # 查找assistant的回答终止符<im_end>
                    if input_ids[end: end+len(self.eos_id)] == self.eos_id:
                        break
                    end += 1
                # 为assistant回答部分(从start+1到end之间)设置loss_mask
                for j in range(start+1, min(end+len(self.eos_id)+1, self.max_length)):
                    loss_mask[j] = 1
                # 跳过到下一个segment
                i = end + len(self.eos_id) if end < len(input_ids) else len(input_ids)
            else:
                i += 1
        return loss_mask
    def __getitem__(self, index):
        sample = self.samples[index]
        # 构建ChatML格式prompt
        prompt = self._create_chat_prompt(sample['conversations'])
        # 分词并截断，确保长度<=max_length
        input_ids = self.tokenizer(prompt).input_ids[:self.max_length]
        # 右侧填充pad_token直到max_length长度
        input_ids += [self.tokenizer.pad_token_id] * (self.max_length - len(input_ids))
        # 生成动态loss_mask, 仅对assistant响应位置计算loss
        loss_mask = self._generate_loss_mask(input_ids)

        # 构建训练样本
        # 模型输入为前n-1个token，预测目标为第2到第n个token
        X = torch.tensor(input_ids[:-1], dtype=torch.long) # 输入序列
        Y = torch.tensor(input_ids[1:], dtype=torch.long) # 目标标签（shifted）
        loss_mask = torch.tensor(loss_mask[1:], dtype=torch.long) # 对齐 Y 的位置（从第一个预测 token 开始）

        return X, Y, loss_mask

d:\miniconda3\envs\minimind\lib\site-packages\torch\cuda\__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


In [3]:
# 构建数据集加载器
from torch.utils.data import DataLoader
from transformers import AutoTokenizer

max_length = 512
data_path = r"D:\MiniMind\dataset\sft_mini_512.jsonl"
tokenizer = AutoTokenizer.from_pretrained(r"D:\MiniMind\model")
train_ds = SFTDataset(data_path, tokenizer, max_length)

train_loader = DataLoader(
    train_ds,
    batch_size=2, # 一个batch有2个样本
    pin_memory=True, # 如果使用GPU，则将数据加载到显存中
    drop_last=False, # 如果最后一批不满batch_size，不丢掉
    shuffle=False, # 不打乱数据顺序
    num_workers=0, # 几个子进程来加载数据，0就是主进程加载,>0就是哆嗦进程并行加载
)

d:\miniconda3\envs\minimind\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
print(len(train_loader))
for item in train_loader:
    print([i.shape for i in item])
    break

607362
[torch.Size([2, 511]), torch.Size([2, 511]), torch.Size([2, 511])]


# 有监督微调

In [ ]:
import torch.nn as nn

loss_fct = nn.CrossEntropyLoss(reduction='none')
for step, (X, Y, loss_mask) in enumerate(train_loader):
    X = X.to(args.device)
    Y = Y.to(args.device)
    loss_mask = loss_mask.to(args.device)
    lr = get_lr(epoch * iter_per_epoch + step, args.epochs * iter_per_epoch, args.learning_rate)

    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
    
    with ctx:
        res = model(X)
        loss = loss_fct(
            res.logits.view(-1, res.logits.size(-1)),
            Y.view(-1)
        ).view(Y.size())

        loss = (loss * loss_mask).sum() / loss_mask.sum()
        loss += res.aux_loss
        loss = loss / args.accumulation_steps
    
    scaler.scale(loss).backward()

    if (step + 1) % args.accumulation_steps == 0:
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), args.grad_clip)

        scaler.step(optimizer)
        scaler.update()

        optimizer.zero_grad(set_to_one=True)

NameError: name 'args' is not defined